In [1]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    # !pip install -q -U tfx==0.15.0rc0
    print("You can safely ignore the package incompatibility errors.")
except Exception:
    pass


import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
print(tf.__version__)

from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

TensorFlow 2.x selected.
You can safely ignore the package incompatibility errors.
2.1.0


In [0]:
input_ = keras.layers.Input(shape=(100,))

emb = keras.layers.Embedding(10, 50)(input_)

conv1 = keras.layers.Conv1D(32, 3, strides=1, padding='same')(emb)

conv2 = keras.layers.Conv1D(32, 3, strides=1, padding='causal')(emb)

In [12]:
emb.get_shape()

TensorShape([None, 100, 50])

In [0]:
model = keras.models.Sequential()
model.add(keras.layers.Embedding(10, 50, input_shape=(100,)))
model.add(keras.layers.Conv1D(32, 3, strides=1, padding='same'))

In [35]:
conv_layer = model.layers[1]

conv_layer.get_weights()[0].shape   # kernel size, last_layer_dim, filtersize

(5, 50, 32)

In [0]:
model = keras.models.Sequential()
model.add(keras.layers.Embedding(10, 50, input_shape=(100,)))
model.add(keras.layers.Flatten())  # 50 * 100
model.add(keras.layers.Dense(32))

In [40]:
conv_layer = model.layers[2]

conv_layer.get_weights()[0] 
conv_layer.get_weights()[0].shape   # last layer output size, units 

(5000, 32)

In [4]:
conv1.get_shape()

TensorShape([None, 100, 32])

In [5]:
conv2.get_shape()

TensorShape([None, 100, 32])

In [0]:
h1, h2 = tf.split(conv2, 2, -1)

In [7]:
h1

<tf.Tensor 'split:0' shape=(None, 100, 16) dtype=float32>

In [8]:
h2

<tf.Tensor 'split:1' shape=(None, 100, 16) dtype=float32>

In [9]:
from keras.datasets import imdb

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=1000)

Using TensorFlow backend.


17465344/17464789 [==============================] - 0s 0us/step


In [0]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen = 1000)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen = 1000)


In [11]:
x_train.shape

(25000, 1000)

In [0]:
def hc(inputs, filters=None, size=1, rate=1, padding='SAME', dropout_rate=0, use_bias=True, activation_fn=None,
       training=True, scope='hc', reuse=None):
    '''
    Args:
      inputs: A 3-D tensor with shape of [batch, time, depth].
      filters: An int. Number of outputs (=activation maps)
      size: An int. Filter size.
      rate: An int. Dilation rate.
      padding: Either `same` or `valid` or `causal` (case-insensitive).
      use_bias: A boolean.
      activation_fn: A string.
      training: A boolean. If True, dropout is applied.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
    Returns:
      A masked tensor of the same shape and dtypes as `inputs`.  <- MASKED TENSOR
    '''
    _input = inputs
    with tf.variable_scope(scope):
        if padding.lower() == 'casual':
            pad_len = (size - 1) * rate
            inputs = tf.pad(inputs, [[0, 0], [pad_len, 0]], [0, 0])
            padding = 'valid'

        if filters is None:
            filters = inputs.get_shape().as_list()[-1]

        params = {"inputs": inputs, "filters": 2 * filters, "kernel_size": size,
                  "dilation_rate": rate, "padding": padding, "use_bias": use_bias,
                  "kernel_initializer": tf.contrib.layers.variance_scaling_initializer(), "reuse": reuse}

        tensor = tf.layers.conv1d(**params)

        h1, h2 = tf.split(tensor, 2, axis=-1)
        h1 = normalize(h1, scope='h1')
        h2 = normalize(h2, scope='h2')

        h1 = tf.nn.sigmoid(h1, 'gate')
        h2 = activation_fn(h2, 'info') if activation_fn is not None else h2
        tensor = h1 * h2 + _input * (1. - h1)
        tensor = tf.layers.dropout(tensor, rate=dropout_rate, trainig=training)
    return tensor

In [58]:
input_ = tf.keras.layers.Input(shape=(1000,))
emb = tf.keras.layers.Embedding(1000+1, 300)(input_)
conv = tf.keras.layers.Conv1D(128, 5, 1, padding='same', activation='relu')(emb)


conv

<tf.Tensor 'conv1d_15/Identity:0' shape=(None, 1000, 128) dtype=float32>

In [0]:
print(tf.convert_to_tensor([[1,2,3]]), '\n')

for i in range(5):
  print(tf.pad(tf.convert_to_tensor([[1,2,3]]), paddings= [[0,0], [i,0]], mode='constant', constant_values=-1))

tf.Tensor([[1 2 3]], shape=(1, 3), dtype=int32) 

tf.Tensor([[1 2 3]], shape=(1, 3), dtype=int32)
tf.Tensor([[-1  1  2  3]], shape=(1, 4), dtype=int32)
tf.Tensor([[-1 -1  1  2  3]], shape=(1, 5), dtype=int32)
tf.Tensor([[-1 -1 -1  1  2  3]], shape=(1, 6), dtype=int32)
tf.Tensor([[-1 -1 -1 -1  1  2  3]], shape=(1, 7), dtype=int32)


In [0]:
conv.shape

TensorShape([None, 1000, 128])

In [0]:
def hc_keras(inputs, filters=None, size=1, rate=1, padding='same', dropout_rate=0, use_bias=True,
             training=True, name='hc_keras'):
    '''
    This layer is not mask
    '''
    if filters is None:
        filters = K.int_shape(inputs)[-1]

    tensor_1 = tf.keras.layers.Conv1D(filters=filters, kernel_size=size, dilation_rate=rate, padding=padding,
                                      use_bias=use_bias, )(inputs)

    tensor_2 = tf.keras.layers.Conv1D(filters=filters, kernel_size=size, dilation_rate=rate, padding=padding,
                                      use_bias=use_bias, bias_initializer=tf.keras.initializers.Constant(-3.0),
                                      )(inputs)

    h1 = tf.keras.layers.Activation('relu')(tensor_1)
    h2 = tf.keras.layers.Activation('sigmoid')(tensor_2)

    left = tf.keras.layers.Multiply()([h1, h2])
    right = tf.keras.layers.Lambda(lambda x: 1.0 - x)(h2)
    right2 = tf.keras.layers.Multiply()([inputs, right])
    output = tf.keras.layers.Add()([left, right2])
    output = tf.keras.layers.Dropout(dropout_rate)(output)
    return output

In [0]:
input_ = tf.keras.layers.Input(shape=(1000,))
emb = tf.keras.layers.Embedding(1000+1, 300)(input_)
conv = tf.keras.layers.Conv1D(128, 5, 1, padding='same', activation='relu')(emb)

for h in range(5):
  conv = hc_keras(conv, name='hc_keras_{}'.format(h))
  h += 1

x = tf.keras.layers.Flatten()(conv)
out_ = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.models.Model(input_, out_)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [0]:
hist = model.fit(x_train, y_train, batch_size=512, epochs=10, validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 68s 3ms/sample - loss: 0.6722 - acc: 0.5191 - val_loss: 0.5735 - val_acc: 0.6770
Epoch 2/10
25000/25000 [==============================] - 59s 2ms/sample - loss: 0.4417 - acc: 0.7993 - val_loss: 0.3879 - val_acc: 0.8283
Epoch 3/10
25000/25000 [==============================] - 59s 2ms/sample - loss: 0.3614 - acc: 0.8434 - val_loss: 0.3842 - val_acc: 0.8358
Epoch 4/10
25000/25000 [==============================] - 59s 2ms/sample - loss: 0.3257 - acc: 0.8600 - val_loss: 0.3534 - val_acc: 0.8490
Epoch 5/10
25000/25000 [==============================] - 59s 2ms/sample - loss: 0.3008 - acc: 0.8726 - val_loss: 0.3363 - val_acc: 0.8554
Epoch 6/10
25000/25000 [==============================] - 59s 2ms/sample - loss: 0.2705 - acc: 0.8894 - val_loss: 0.3298 - val_acc: 0.8595
Epoch 7/10
25000/25000 [==============================] - 59s 2ms/sample - loss: 0.2464 - acc: 0.9004 - val_loss: 0.